In [11]:
import os
import zipfile
import glob
from pathlib import Path

In [12]:

directory_path = './content/train'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' was created.")
else:
    print(f"Directory '{directory_path}' already exists.")

directory_path = './content/test'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' was created.")
else:
    print(f"Directory '{directory_path}' already exists.")

Directory './content/train' was created.
Directory './content/test' was created.


In [13]:
with zipfile.ZipFile('Training.zip', 'r') as zip_ref:
    zip_ref.extractall('./content/train')
with zipfile.ZipFile('Testing.zip', 'r') as zip_ref:
    zip_ref.extractall('./content/test')

In [14]:
path = ""
def label_glioma(folder):
  for i,f in enumerate(folder):
    os.rename(f, os.path.join(path+"/glioma_tumor", 'glioma_'+'{0:03d}.jpg'.format(i)))
  glioma = glob.glob(path+"glioma_tumor"+'/*')
  print("{}files changed".format(i+1))
def label_meningioma(folder):
  for i,f in enumerate(folder):
    os.rename(f, os.path.join(path+"/meningioma_tumor", 'meningioma_'+'{0:03d}.jpg'.format(i)))
  glioma = glob.glob(path+"meningioma_tumor"+'/*')
  print("{}files changed".format(i+1))
def label_no(folder):
  for i,f in enumerate(folder):
    os.rename(f, os.path.join(path+"/no_tumor", 'no_'+'{0:03d}.jpg'.format(i)))
  glioma = glob.glob(path+"no_tumor"+'/*')
  print("{}files changed".format(i+1))
def label_pituitary(folder):
  for i,f in enumerate(folder):
    os.rename(f, os.path.join(path+"/pituitary_tumor", 'pituitary_'+'{0:03d}.jpg'.format(i)))
  glioma = glob.glob(path+"pituitary_tumor"+'/*')
  print("{}files changed".format(i+1))

In [15]:
path = "./content/train/Training"
glioma = glob.glob(path+"/glioma_tumor"+'/*.jpg')
meningioma = glob.glob(path+"/meningioma_tumor"+"/*.jpg")
no = glob.glob(path+"/no_tumor"+"/*.jpg")
pituitary = glob.glob(path+"/pituitary_tumor"+"/*.jpg")

label_glioma(glioma)
label_meningioma(meningioma)
label_no(no)
label_pituitary(pituitary)

path = "./content/test/Testing"
glioma = glob.glob(path+"/glioma_tumor"+'/*.jpg')
meningioma = glob.glob(path+"/meningioma_tumor"+"/*.jpg")
no = glob.glob(path+"/no_tumor"+"/*.jpg")
pituitary = glob.glob(path+"/pituitary_tumor"+"/*.jpg")

label_glioma(glioma)
label_meningioma(meningioma)
label_no(no)
label_pituitary(pituitary)

826files changed
822files changed
395files changed
827files changed
100files changed
115files changed
105files changed
74files changed


In [16]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import json
from PIL import Image
import random
import gzip
import pickle

save_train_pt_path = "./content/train/train_datasets"
save_test_pt_path = "./content/test/test_datasets"

In [17]:
def image_to_tensor(path) -> torch.Tensor:
    resize_trans = transforms.Compose([transforms.Resize((512,512)), transforms.Grayscale(), transforms.ToTensor()])
    image = Image.open(path)
    image_tensor = resize_trans(image)
    return image_tensor

In [18]:
def save_images(path, save_path: str):
    
    listed_image_paths = []
    for folder in os.listdir(path):
        for filename in os.listdir(path + "/{}".format(folder)):
            image_path = path + "/{}".format(folder) + "/{}".format(filename)
            listed_image_paths.append(image_path)
    
    random.shuffle(listed_image_paths)
    
    IO_pairs = []
    set = 1
    for i, path_str in enumerate(listed_image_paths):
        tensor = image_to_tensor(path_str)
        path = Path(path_str)
        filename = path.parts[len(path.parts) - 1]

        if filename.startswith('g'):
            image_class_name='glioma'
            tensor_class_target = torch.tensor([1,0,0,0])

        if filename.startswith('m'):
            image_class_name='meningioma'
            tensor_class_target = torch.tensor([0,1,0,0])

        if filename.startswith('p'):
            image_class_name='pituitary'
            tensor_class_target = torch.tensor([0,0,1,0])

        if filename.startswith('n'):
            image_class_name='no'
            tensor_class_target = torch.tensor([0,0,0,1])
        
        example = [image_class_name, tensor.tolist(), tensor_class_target]
        IO_pairs.append(example)

        if (len(IO_pairs) == 1000 or i + 1 == len(listed_image_paths)):
            dataset = ImageDataset(IO_pairs)
            print(dataset.__len__())
            torch.save(dataset, save_path + "/{}_dataset{}.pt".format("train" if "train" in save_path else "test", set))
            IO_pairs = []
            set += 1

class ImageDataset(Dataset):
    def __init__(self, IO_pairs):
        self.IO_pairs = IO_pairs

    def __len__(self):
        return len(self.IO_pairs)

    def __getitem__(self, index):
        # Get the image
        image_class_name = self.IO_pairs[index][0]
        image_tensor_list = self.IO_pairs[index][1]
        classification_target = self.IO_pairs[index][2]

        return image_class_name, image_tensor_list, classification_target

In [19]:
# We are given the glioma_tumor folder
# We want a list that looks like this: [[image_class_name, image_tensor_list, classification_target], 
#                                       [image_class_name, image_tensor_list, classification_target], ... ]

# Where do we get our content from? The input to the save_images function
train_path = "./content/train/Training"
test_path = "./content/test/Testing"

# Where do we store our pt files? Output of the save_images function
save_train_pt_path = "./content/train/train_datasets"
save_test_pt_path = "./content/test/test_datasets"

if not os.path.exists(save_train_pt_path):
    os.mkdir(save_train_pt_path)
if not os.path.exists(save_test_pt_path):
    os.mkdir(save_test_pt_path)

save_images(train_path, save_train_pt_path)
save_images(test_path, save_test_pt_path)

1000
1000
870
394


In [20]:
# Torch loading demo
train_paths = []
for file in os.listdir(save_train_pt_path):
    train_paths.append(save_train_pt_path + "/{}".format(file))

loaded_data = torch.load(train_paths[0])
print(type(loaded_data))

<class '__main__.ImageDataset'>


In [23]:
accum = 0
for i in range(loaded_data.__len__()):
    if loaded_data.__getitem__(i)[0] == 'meningioma':
        accum += 1

print(accum)

279


In [36]:
print("Random class: " + loaded_data.__getitem__(5)[0])

Random class: pituitary
